<a href="https://colab.research.google.com/github/vishtantrik/flight-poster/blob/data_prep_notebook/Explore_and_Prepare_Data_for_Visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>